 # NBA 디트로이트 팀 성과 분석 실습



 ## 학습 목표

 - "문제 → 검색 → 적용 → 검증" 프로세스 실습

 - AI 답변을 그대로 복붙하지 않고 단계별로 이해하며 구현

 - 같은 결과를 얻는 다양한 방법 탐색

 - 직접 검증을 통한 결과 확인



 ## 분석 목표

 1. 디트로이트 팀의 전체 게임 승률과 리그 평균 승률 비교

 2. 디트로이트 팀의 클러치 게임 승률과 하위 5개 팀 승률 비교

 3. 분석 결과 시각화

 ---

 # 실습 1: NBA 디트로이트 팀 성과 분석

 ---

 ## 1단계: 문제 정의 및 분해

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import platform

# OS 자동 감지 및 한글 폰트 설정
system = platform.system()

if system == 'Windows':
    # Windows: 맑은 고딕 사용
    plt.rcParams['font.family'] = 'Malgun Gothic'
    
elif system == 'Darwin':  # Darwin = macOS
    # macOS: 애플 고딕 사용
    plt.rcParams['font.family'] = 'AppleGothic'
    
elif system == 'Linux':
    # Linux: 나눔 고딕 사용 (사전 설치 필요)
    # 터미널: sudo apt-get install -y fonts-nanum
    plt.rcParams['font.family'] = 'NanumGothic'
    
else:
    print(f"알 수 없는 OS: {system}")

# 모든 OS 공통: 음수 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

print(f"OS: {system}, 설정된 폰트: {plt.rcParams['font.family']}")

OS: Darwin, 설정된 폰트: ['AppleGothic']


 ### 문제 1-1: 분석 문제를 논리적 단계로 분해하기



 "디트로이트 팀의 성과 분석"이라는 큰 문제를 해결 가능한 작은 단계로 나누세요.

In [7]:
# 전체 분석 단계
"""
1단계: 데이터 로드 및 구조 파악
2단계: 데이터 전처리
3단계: 데이터 탐색
4단계: 모든 팀 승률 계산 (GroupBy, Pivot Table 방법)
5단계: 검증을 위한 디트로이트 팀 승률 직접 계산
6단계: 전체 승률 비교 시각화
7단계: 클러치 게임 식별 및 분석 (과제)
8단계: 클러치 게임 결과 시각화 (과제)
"""

'\n1단계: 데이터 로드 및 구조 파악\n2단계: 데이터 전처리\n3단계: 데이터 탐색\n4단계: 모든 팀 승률 계산 (GroupBy, Pivot Table 방법)\n5단계: 검증을 위한 디트로이트 팀 승률 직접 계산\n6단계: 전체 승률 비교 시각화\n7단계: 클러치 게임 식별 및 분석 (과제)\n8단계: 클러치 게임 결과 시각화 (과제)\n'

 ## 2단계: 데이터 로드 및 전처리

 ### 문제 2-1: 데이터 로드

In [8]:
# 필요한 데이터 로드
games = pd.read_csv("../data/games.csv")
teams = pd.read_csv("../data/teams.csv")

print("데이터 로드 완료")
print(f"games shape: {games.shape}")
print(f"teams shape: {teams.shape}")

데이터 로드 완료
games shape: (26651, 21)
teams shape: (30, 14)


 ### 문제 2-2: 데이터 구조 파악

In [9]:
# games 데이터의 정보 확인
print("Games 데이터 정보:")
print(games.info())
print("\n첫 5행:")
display(games.head())


Games 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26651 entries, 0 to 26650
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GAME_DATE_EST     26651 non-null  object 
 1   GAME_ID           26651 non-null  int64  
 2   GAME_STATUS_TEXT  26651 non-null  object 
 3   HOME_TEAM_ID      26651 non-null  int64  
 4   VISITOR_TEAM_ID   26651 non-null  int64  
 5   SEASON            26651 non-null  int64  
 6   TEAM_ID_home      26651 non-null  int64  
 7   PTS_home          26552 non-null  float64
 8   FG_PCT_home       26552 non-null  float64
 9   FT_PCT_home       26552 non-null  float64
 10  FG3_PCT_home      26552 non-null  float64
 11  AST_home          26552 non-null  float64
 12  REB_home          26552 non-null  float64
 13  TEAM_ID_away      26651 non-null  int64  
 14  PTS_away          26552 non-null  float64
 15  FG_PCT_away       26552 non-null  float64
 16  FT_PCT_away       26552 no

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,...,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,...,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0


In [10]:
# teams 데이터의 정보 확인
print("Teams 데이터 정보:")
print(teams.info())
print("\n첫 5행:")
display(teams.head())


Teams 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   LEAGUE_ID           30 non-null     int64  
 1   TEAM_ID             30 non-null     int64  
 2   MIN_YEAR            30 non-null     int64  
 3   MAX_YEAR            30 non-null     int64  
 4   ABBREVIATION        30 non-null     object 
 5   NICKNAME            30 non-null     object 
 6   YEARFOUNDED         30 non-null     int64  
 7   CITY                30 non-null     object 
 8   ARENA               30 non-null     object 
 9   ARENACAPACITY       26 non-null     float64
 10  OWNER               30 non-null     object 
 11  GENERALMANAGER      30 non-null     object 
 12  HEADCOACH           30 non-null     object 
 13  DLEAGUEAFFILIATION  30 non-null     object 
dtypes: float64(1), int64(5), object(8)
memory usage: 3.4+ KB
None

첫 5행:


,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends


 ### 문제 2-3: 데이터 품질 확인

 - 결측치, 중복 데이터 확인

In [11]:
# 결측치 확인
print("=== Games 데이터 결측치 확인 ===")
print(games.isnull().sum())
print(f"\n전체 행 수: {len(games)}")

print("\n=== Teams 데이터 결측치 확인 ===")
print(teams.isnull().sum())
print(f"\n전체 행 수: {len(teams)}")


=== Games 데이터 결측치 확인 ===
GAME_DATE_EST        0
GAME_ID              0
GAME_STATUS_TEXT     0
HOME_TEAM_ID         0
VISITOR_TEAM_ID      0
SEASON               0
TEAM_ID_home         0
PTS_home            99
FG_PCT_home         99
FT_PCT_home         99
FG3_PCT_home        99
AST_home            99
REB_home            99
TEAM_ID_away         0
PTS_away            99
FG_PCT_away         99
FT_PCT_away         99
FG3_PCT_away        99
AST_away            99
REB_away            99
HOME_TEAM_WINS       0
dtype: int64

전체 행 수: 26651

=== Teams 데이터 결측치 확인 ===
LEAGUE_ID             0
TEAM_ID               0
MIN_YEAR              0
MAX_YEAR              0
ABBREVIATION          0
NICKNAME              0
YEARFOUNDED           0
CITY                  0
ARENA                 0
ARENACAPACITY         4
OWNER                 0
GENERALMANAGER        0
HEADCOACH             0
DLEAGUEAFFILIATION    0
dtype: int64

전체 행 수: 30


In [12]:
# 중복 데이터 확인
print("=== 중복 데이터 확인 ===")
# 고유값 GAME_ID 기준으로 확인
duplicate_games = games.duplicated(subset=['GAME_ID']).sum()
print(f"Games 데이터 중복 행: {duplicate_games}")

# 고유값 TEAM_ID 기준으로 확인
duplicate_teams = teams.duplicated(subset=['TEAM_ID']).sum()
print(f"Teams 데이터 중복 행: {duplicate_teams}")


=== 중복 데이터 확인 ===
Games 데이터 중복 행: 29
Teams 데이터 중복 행: 0


 ### 문제 2-4: 데이터 전처리

In [13]:
# Step 1: 중복 제거
before_rows = len(games)
games = games.drop_duplicates(subset=['GAME_ID'])
after_rows = len(games)
print(f"제거된 중복 행: {before_rows - after_rows}")


제거된 중복 행: 29


In [14]:
# Step 2: 결측치 처리 - 승패 정보가 없는 게임 확인
# HOME_TEAM_WINS가 결측인 경우 확인
missing_wins = games[games['HOME_TEAM_WINS'].isnull()]
print(f"승패 정보가 없는 게임: {len(missing_wins)}개")


승패 정보가 없는 게임: 0개


In [15]:
# Step 3: 점수 데이터 확인 및 처리
# 점수 컬럼 결측치 확인
score_missing = games[['PTS_home', 'PTS_away']].isnull().sum()
print("점수 결측치:")
print(score_missing)

# 점수가 없는 게임 확인
games_no_score = games[(games['PTS_home'].isnull()) | (games['PTS_away'].isnull())]
print(f"\n점수가 없는 게임: {len(games_no_score)}개")

if len(games_no_score) > 0:
    # 클러치 게임 분석을 위해 점수가 필요하므로 해당 게임 제거
    games_with_score = games.dropna(subset=['PTS_home', 'PTS_away'])
    print(f"점수가 있는 게임만 필터링: {len(games_with_score)}개")
    games = games_with_score


점수 결측치:
PTS_home    99
PTS_away    99
dtype: int64

점수가 없는 게임: 99개
점수가 있는 게임만 필터링: 26523개


 ### 문제 2-5: 파생 변수 생성

In [16]:
# Step 1: games테이블의 팀 이름에 대한 컬럼 생성
# - teams 테이블의 도시명 활용
# map : 함수로 값을 새롭게 생성

# 디트로이트 팀의 id를 확인
teams[teams['CITY'].str.contains('Detroit', case=False)]

detroit_id = teams[teams['CITY'].str.contains('Detorit', case = False)]['TEAM_ID']
print (detroit_id)

Series([], Name: TEAM_ID, dtype: int64)


 ## 3단계: 데이터 탐색

 ### 문제 3-1: 주요 컬럼 이해하기

In [ ]:
# 팀 도시 이름 매핑 - 딕셔너리
# ID를 키값, City 값으로 지정
team_mapping = dict(zip(teams['TEAM_ID'], teams['CITY']))
team_mapping

games['HOME_TEAM'] = games['HOME_TEAM_ID'].map(team_mapping)


{1610612737: 'Atlanta',
 1610612738: 'Boston',
 1610612740: 'New Orleans',
 1610612741: 'Chicago',
 1610612742: 'Dallas',
 1610612743: 'Denver',
 1610612745: 'Houston',
 1610612746: 'Los Angeles',
 1610612747: 'Los Angeles',
 1610612748: 'Miami',
 1610612749: 'Milwaukee',
 1610612750: 'Minnesota',
 1610612751: 'Brooklyn',
 1610612752: 'New York',
 1610612753: 'Orlando',
 1610612754: 'Indiana',
 1610612755: 'Philadelphia',
 1610612756: 'Phoenix',
 1610612757: 'Portland',
 1610612758: 'Sacramento',
 1610612759: 'San Antonio',
 1610612760: 'Oklahoma City',
 1610612761: 'Toronto',
 1610612762: 'Utah',
 1610612763: 'Memphis',
 1610612764: 'Washington',
 1610612765: 'Detroit',
 1610612766: 'Charlotte',
 1610612739: 'Cleveland',
 1610612744: 'Golden State'}

 ## 4단계: 모든 팀 승률 계산 (GroupBy, Pivot Table)

- 4-1: GroupBy 
    - 직접 단계적 풀이 도출 또는 AI 검색으로 도출

- 4-2: Pivot Table (과제)
    - 직접 단계적 풀이 도출 또는 AI 검색으로 도출

 ### 문제 4-1: GroupBy를 사용한 승률 계산

In [17]:
# Step 1: 홈팀 기준 집계


In [18]:
# Step 2: 원정팀 기준 집계


In [19]:
# Step 3: 홈과 원정 통합


In [20]:
# Step 4: 정렬


 ### 문제 4-2: Pivot Table을 사용한 승률 계산 (제공)

In [21]:
# Step 1: 홈 게임 집계
home_stats = games.pivot_table(
    index='HOME_TEAM',
    values='HOME_TEAM_WINS',
    aggfunc=['sum', 'count']
)
home_stats.columns = ['home_wins', 'home_games']
print("홈 게임 집계 완료")

# Step 2: 원정 게임 집계 (승패 반전)
games['AWAY_WINS'] = 1 - games['HOME_TEAM_WINS']
away_stats = games.pivot_table(
    index='VISITOR_TEAM',
    values='AWAY_WINS',
    aggfunc=['sum', 'count']
)
away_stats.columns = ['away_wins', 'away_games']
print("원정 게임 집계 완료")

# Step 3: 인덱스 기준 병합
team_stats_pivot = home_stats.join(away_stats, how='outer').fillna(0)
team_stats_pivot['total_wins'] = team_stats_pivot['home_wins'] + team_stats_pivot['away_wins']
team_stats_pivot['total_games'] = team_stats_pivot['home_games'] + team_stats_pivot['away_games']
team_stats_pivot['winrate'] = team_stats_pivot['total_wins'] / team_stats_pivot['total_games']

# 정렬 및 인덱스 리셋
team_stats_pivot = team_stats_pivot.sort_values('winrate', ascending=False).reset_index()
team_stats_pivot.rename(columns={'HOME_TEAM': 'team'}, inplace=True)
    
print("\n완료! 상위 3팀:")
display(team_stats_pivot.head(3))

KeyError: 'HOME_TEAM'

 ### 문제 4-3: 두 방법 결과 비교

In [ ]:
# GroupBy와 Pivot Table 결과 비교
comparison = pd.merge(
    team_stats_df[['team', 'winrate']].rename(columns={'winrate': 'groupby_winrate'}),
    team_stats_pivot[['team', 'winrate']].rename(columns={'winrate': 'pivot_winrate'}),
    on='team'
)

comparison['difference'] = abs(comparison['groupby_winrate'] - comparison['pivot_winrate'])

print("두 방법의 승률 차이 확인:")
print(f"최대 차이: {comparison['difference'].max():.10f}")
print(f"평균 차이: {comparison['difference'].mean():.10f}")
print(f"모든 값이 동일: {comparison['difference'].max() < 1e-10}")

print("\n비교 샘플 (상위 5팀):")
display(comparison.head())

 ## 5단계: 검증을 위한 디트로이트 팀 승률 직접 계산

- 5단계: 검증을 위한 디트로이트 팀 승률 계산
    - 직접 도출

 ### 문제 5-1: 홈 게임 분석

In [ ]:
# Step 5-1-1: 디트로이트가 홈팀인 게임 찾기


In [ ]:
# Step 5-1-2: 홈 게임 승리 수 계산


In [ ]:
# Step 5-1-3: 홈 승률 계산


 ### 문제 5-2: 원정 게임 분석

In [ ]:
# Step 5-2-1: 디트로이트가 원정팀인 게임 찾기


In [ ]:
# Step 5-2-2: 원정 게임 승패 판단


In [ ]:
# Step 5-2-3: 원정 승률 계산

 ### 문제 5-3: 전체 승률 계산

In [ ]:
# Step 5-3-1: 홈과 원정 통합


In [ ]:
# Step 5-3-2: 전체 승률 계산


In [ ]:
# Step 5-3-3: 검증 - GroupBy 결과와 비교


 ### 문제 5-4: 리그 내 위치 분석

In [ ]:
# Step 5-4-1: 리그 평균 승률 계산


In [ ]:
# Step 5-4-2: 디트로이트 순위 확인


In [ ]:
# Step 5-4-3: 디트로이트와 리그 평균 비교


 ## 6단계: 전체 승률 비교 시각화

 ### 문제 6-1: 시각화 데이터 준비

In [ ]:
# Step 6-1-1: 필요한 데이터 추출


In [ ]:
# Step 6-1-2: 그래프용 데이터 구성


 ### 문제 6-2: 막대 그래프 생성

In [ ]:
# 그래프 생성
plt.figure(figsize=(12, 6))

# 색상 설정 (디트로이트 강조)
colors = ['#FF4444',  # 디트로이트 - 빨간색
          '#666666',  # 리그 평균 - 회색
          '#4CAF50', '#4CAF50', '#4CAF50',  # 상위팀 - 초록색
          '#FFC107', '#FFC107', '#FFC107']  # 하위팀 - 노란색

# 막대 그래프 생성
bars = plt.bar(teams_labels, winrates, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)

# 제목 및 레이블
plt.title('NBA 팀별 승률 비교 (디트로이트 vs 리그 평균 vs 상하위팀)', 
          fontsize=16, fontweight='bold', pad=20)
plt.ylabel('승률', fontsize=12)
plt.xlabel('팀', fontsize=12)

# 막대 위에 값 표시
for bar, rate in zip(bars, winrates):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{rate:.3f}\n({rate*100:.1f}%)',
             ha='center', va='bottom', fontsize=10, fontweight='bold')

# 그래프 꾸미기
plt.grid(True, axis='y', alpha=0.3, linestyle='--')
plt.ylim(0, max(winrates) * 1.1)

# 범례 추가
legend_elements = [
    plt.Rectangle((0,0),1,1, color='#FF4444', alpha=0.8, label='디트로이트'),
    plt.Rectangle((0,0),1,1, color='#666666', alpha=0.8, label='리그 평균'),
    plt.Rectangle((0,0),1,1, color='#4CAF50', alpha=0.8, label='상위 3팀'),
    plt.Rectangle((0,0),1,1, color='#FFC107', alpha=0.8, label='하위 3팀')
]
plt.legend(handles=legend_elements, loc='upper right')

# x축 레이블 회전
plt.xticks(rotation=45, ha='right')

# 레이아웃 조정 및 표시
plt.tight_layout()
plt.show()

 ---

 # 과제

 ---

 ## 7단계: 클러치 게임 식별 및 분석 (과제)



 ### 과제 내용

 1. 클러치 게임 정의: 점수 차이가 5점 이하인 게임

 2. 디트로이트의 클러치 게임 승률 계산

 3. 모든 팀의 클러치 게임 승률 계산

 4. 디트로이트와 하위 5팀 비교



 ### 힌트

 - 점수 차이: `abs(games['PTS_home'] - games['PTS_away'])`

 - 조건부 승패 판단: `np.where()` 활용

 - 클러치 게임에서도 홈/원정 구분 필요

In [ ]:
# TODO: 7단계 과제를 구현하세요


 ## 8단계: 클러치 게임 결과 시각화 (과제)



 ### 과제 내용

 1. 클러치 승률 비교 막대 그래프 (디트로이트 vs 하위 5팀)



 ### 요구 사항

 - 디트로이트는 다른 색으로 강조

In [ ]:
# TODO: 8단계 과제를 구현하세요


 ## 분석 결과 요약 정리